# S-2 image processing for NDVI retrieval from GEE
This code is a semi-automatic routine for the download of NDVI values from Sentinel-2 via the Google Earth Engine API. The downloaded data represent the mean value of NDVI over an area of interest (AoI). The shape of the AoI must be provided manually into the code as a Java polygon or similar object.


- The full timeseries of data is downloaded (from april 2017 to today).
- All the images are reprojected with respect to the first one of the collection, that is the same as performing coregistration of all images.
- A threshold-based mask for cloud covering is applied to the image such that any pixels with a probability value less than or equal to the value of the threshold will be kept, while all other pixels will be removed. The updated image with the applied mask is then returned. The threshold is user-defined by input.
- The database of NDVI values is resampled to daily frequency and linearly interpolated.

**Output**

The output database has columns:
- Date: timestamp of date and time of passage rounded at day.
- NDVI: NDVI values, [dimensionless]. NDVI has range [-1,1]: negative values are related to clouds or water, positive small values (near 0) to bare soil, higher positive values to healthy vegetation.
- Geometry: name of AoI.

---
**Dependencies** 

This code requires the installation of the Earth Engine API, `ee`. You can find more info on the installation procedure here: [Python installation of GEE](https://developers.google.com/earth-engine/guides/python_install). \
This code runs on browser-based notebooks only (Google Colaboratory, Jupyter Notebooks, etc...). \
Be aware that you won't need to install the Google Cloud APK to run the code.

The full dependencies required are provided in the file of the environment google, `google.yml`. To install the virtual environment by using conda, run the lines below in the terminal after having activated `conda`.

```python
conda env create -f environment.yml # install environment from file environment.yml
conda activate <myenv> # activate environment with name provided by header of .yml file
conda info --envs # to check if everything worked fine
```
If you don't use `conda`, manually install the dependencies listed in the environment file by using `pip`.

---

In [1]:
from funcs_gee import *

In [2]:
# Extract data

def maskS2clouds(image):
    qa = image.select('QA60') # 60 m spat res TO0 BIG!!!
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
        qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask) \
                .select("B.*") \
                .copyProperties(image, ["system:time_start"])

def maskCloudPerc(image):
    global thr
    return image.updateMask(image.select('probability').lte(thr))

def addNDVI(image:ee.Image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

def extract_NDVI(image:ee.Image):
    """Ausiliary function to extract data from an Image
    
    Optimal implementation is to map this function
    on a whole ImageCollection via .map() and insert the
    return into a ee.FeatureCollection.
    
    Return
    ------
    ee.Feature
    
    """
    
    ndvi=image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    image=image.addBands(ndvi)
    
    mean = image.reduceRegion(**{ 
        'reducer': ee.Reducer.mean(),
        'geometry': aoi,
    })
        
    properties = {
        'Date': image.get('system:time_start'), # only way to get a timestr is an external operation
        'Geometry': geometry_title,
        'NDVI': mean.get('NDVI'),
    }
    return ee.Feature(None, properties)


def clean_date(date:int):
    return time.strftime('%x %H', time.localtime((date)/1000))

In [3]:
# Trigger the authentication flow.
ee.Authenticate()
 
# Initialize the library.
ee.Initialize()

In [4]:
print('Define area of interest. \nIf you have a GeoJSON file, copy paste.\n'+
      'If you have a KML, export to GeoJSON (plenty of free tools online).')

geoJSON = {
"type": "FeatureCollection",
"name": "merged",
"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
"features": [
{ "type": "Feature", "properties": { "Name": "Budrio_campo_safe_half", "description": None, "tessellate": 1 }, "geometry": { "type": "Polygon", "coordinates": [ [ [ 11.53262979564736, 44.570842547510622 ], [ 11.532328100248961, 44.570445732016537 ], [ 11.53264162483709, 44.570339694294631 ], [ 11.532950828277439, 44.570738040751841 ] ] ] } } ] }

nfeatures = len(geoJSON['features'])
coords_geojson = [geoJSON['features'][i]['geometry']['coordinates'] for i in range(nfeatures)]
geometry_title=input('Please provide title for geometry: ')

Define area of interest. 
If you have a GeoJSON file, copy paste.
If you have a KML, export to GeoJSON (plenty of free tools online).


Please provide title for geometry:  big


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import netCDF4 as nc

In [12]:
# meters = cos(degree latitude) * (1852/60) * arc-seconds
# 1 arcs = 1/3600 deg
res_m=10
coords_clean=coords_geojson[0][0]
lon=np.unique([coords_clean[i][0] for i in range(len(coords_clean))])
lat=np.unique([coords_clean[i][1] for i in range(len(coords_clean))])
mean_lat=np.mean(lat)
res_degs=res_m/(np.cos(np.deg2rad(mean_lat))*1852/60)/3600 #resolution in degrees

lons=np.arange(min(lon), max(lon), res_degs)
lats=np.arange(min(lat), max(lat), res_degs)
lons_2d, lats_2d = np.meshgrid(lons, lats)
lons_1d, lats_1d=[np.ravel(lons_2d), np.ravel(lats_2d)]
id_2d=np.reshape(np.arange(len(np.ravel(lons_2d))), np.shape(lons_2d))

points_dict={id: { 'id':id, 'coords':(lon, lat) } for id,lon,lat in zip(np.ravel(id_2d),lons_1d, lats_1d)}
points_list=[ points_dict[id]['coords'] for id in points_dict]

for id in points_dict:
    p=points_dict[id]
    lon=(p['coords'][0])
    lat=(p['coords'][1])
    area=[lon-res_degs, lat-res_degs, lon+res_degs, lat+res_degs]
    p['area']=area
    pixel=ee.Geometry.Rectangle(area)

In [7]:
# Filters definition
# sp17 = ee.Filter.date('2017-04-04', '2017-05-22')
# su17 = ee.Filter.date('2017-05-22', '2017-09-15')
# au17 = ee.Filter.date('2017-09-15', '2017-11-02')
tot17 = ee.Filter.date('2017-01-01', '2017-12-31')

aoi = ee.Geometry.MultiPolygon(coords_geojson)

# Get collection of images and filter
s2 = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        #.filter(tot17)
        #.filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE',70))
        .select(['B8', 'B4'])
        .sort('system:time_start'))
clouds = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        #.filter(tot17)
        .sort('system:time_start'))
s2 = s2.combine(clouds)

# acq_times = img.aggregate_array('system:time_start').getInfo()
# len([time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times])

# To get all bands and infos of img collection, use
# img.getInfo()

In [8]:
# Reprojection of images
# This action is equivalent to performing coregistration of the images
# to ensure that the coordinates are coherent in the whole collection

proj=s2.first().projection();
scale_mod=ee.Number(proj.nominalScale()).getInfo()
trans_mod=proj.getInfo()['transform'];
crs_mod=proj.getInfo()['crs'];

s2 = s2.map(lambda image: image.reproject(crs_mod,trans_mod))

In [9]:
# Clouds filtering
thr = 100-int(input('Please provide threshold for cloud masking [number, int]. Ex. \'50\' corresponds to filtering out all images that have 50% or more of pixels that are flagged as covered by clouds.'))
s2_filt = s2.map(maskCloudPerc)

# Add NDVI and extract data
aoi=pixel
data = ee.FeatureCollection(s2_filt.map(extract_NDVI))
data_out = data.getInfo()
data_out_to_df = [e.get('properties') for e in data_out.get('features')]
print('\nCheck data extraction: if something is printed below and has reasonable values, everything is ok (note: date is in a Matlab-like numeric, weird format, don\'t worry.)'); data_out_to_df[0]

Please provide threshold for cloud masking [number, int]. Ex. '50' corresponds to filtering out all images that have 50% or more of pixels that are flagged as covered by clouds. 100



Check data extraction: if something is printed below and has reasonable values, everything is ok (note: date is in a Matlab-like numeric, weird format, don't worry.)


{'Date': 1493028680576, 'Geometry': 'big', 'NDVI': None}

In [10]:
df = pd.DataFrame.from_dict(data_out_to_df)
df.Date = df.Date.apply(lambda x : pd.Timestamp(pd.to_datetime(clean_date(x))))
df = df.groupby('Date',as_index=False).mean()
# df['Data'] = df.Date.apply(lambda x : pd.Timestamp(x.date()))
df = df.set_index('Date')

TypeError: agg function failed [how->mean,dtype->object]

In [ ]:
plt.plot(df.loc['2020'].NDVI, marker='o',linestyle='-')

In [ ]:
df_daily = df.resample('D').asfreq().interpolate(method='linear'); df_daily.info()

In [ ]:
len(df_daily)/len(df)

In [ ]:
opt_interp = True if input('Want to resample at a hourly frequency and interpolate values? [y/n]')=='y' else False
if opt_interp:
    df = df.resample('1H').asfreq().interpolate(method='linear').resample('D').mean()
df['Geometry'] = geometry_title
df

In [ ]:
plt.plot(df.NDVI); plt.title('NDVI timeseries - 2017 to today')

In [ ]:
save = input("Wanna save in Data directory? [y/n] ")
if save=='y':
    opt_name = input('Please provide name of file, without extension: [default: geometry-name_NDVI]')
    if opt_name=='': opt_name=geometry_title+'_NDVI'
    df.to_csv(f'..\\..\\Data\\{opt_name}.csv', sep = '\t')

In [ ]:
# Define the time range.
start_date = '2023-01-01'
end_date = '2023-12-31'

# Define your geometry, with coordinates [lon_min, lat_min, lon_max, lat_max]
region_name = 'region'
coords = [11.532328100248961, 44.570339694294631, 11.532950828277439 , 44.570738040751841]

# ----------------------------------------------------------------------

import ee
import datetime as dt

# Function to calculate NDVI.
def addNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)


# Initialize the Earth Engine module.
ee.Initialize()

region = ee.Geometry.Rectangle(coords)

# Load the Sentinel-2 SR Harmonized collection.
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .filterBounds(region)

proj=s2.first().select('B2').projection()
scale_mod=ee.Number(proj.nominalScale()).getInfo()
trans_mod=proj.getInfo()['transform']
crs_mod=proj.getInfo()['crs']

# Update masks/add bands over the collection.
s2_ndvi = s2.map(addNDVI)

# Get pixels in the region with all bands, lon, lat and time
ndvi = np.array(s2_ndvi.getRegion(region, scale_mod).getInfo())

# Build dataframe and clean timestamps
ndvi_df = pd.DataFrame(ndvi[1:], columns=ndvi[0])
ndvi_df['datetime'] = [dt.datetime.fromtimestamp(ts).strftime("%Y-%m-%d %H:%M") for ts in ndvi_df['time']/1000]
drop_columns=['AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60','id','time']
ndvi_df.drop(drop_columns, axis=1, inplace=True)
ndvi_df=ndvi_df.set_index('datetime')

answ=input('Save csv file? [[y]/n]')
if answ=='y' or answ=='':
    ndvi_df.to_csv(f'NDVI_{start_date}_{end_date}.csv')